In [1]:
import sqlite3
import pandas as pd
import import_ipynb
from analysisMagic import dfm3

<string>:1: DtypeWarning: Columns (3,7,12,16,20,23,25,26,31,33,34,36,38,40,41,42,48,53,54,61,62,66,68) have mixed types. Specify dtype option on import or set low_memory=False.


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 864 entries, 0 to 863
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   setName      864 non-null    object
 1   setCode      864 non-null    object
 2   releaseDate  864 non-null    object
 3   releaseYear  864 non-null    int64 
dtypes: int64(1), object(3)
memory usage: 27.1+ KB


In [2]:
# creates db if it doesn't already exist
conn = sqlite3.connect("../sql/magic.db")

In [3]:
dfm3.head()

,colors,language,name,rarity,setCode,types,uuid,setName,releaseDate,releaseYear,cardFinish,currency,date,gameAvailability,price,priceProvider,providerListing,avgMarketPrice
0,W,English,Ancestor's Chosen,uncommon,10E,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,Tenth Edition,2007-07-13,2007,normal,USD,2025-08-26,paper,0.25,tcgplayer,retail,0.28
1,W,English,Ancestor's Chosen,uncommon,10E,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,Tenth Edition,2007-07-13,2007,foil,USD,2025-08-26,paper,0.52,tcgplayer,retail,0.52
2,W,English,Ancestor's Chosen,uncommon,10E,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,Tenth Edition,2007-07-13,2007,normal,USD,2025-08-26,paper,0.35,cardkingdom,retail,0.28
3,W,English,Ancestor's Chosen,uncommon,10E,Creature,5f8287b1-5bb6-5f4c-ad17-316a40d5bb0c,Tenth Edition,2007-07-13,2007,normal,USD,2025-08-26,paper,0.23,cardsphere,retail,0.28
4,W,English,Ancestor's Chosen,uncommon,10E,Creature,b7c19924-b4bf-56fc-aa73-f586e940bd42,Tenth Edition,2007-07-13,2007,normal,USD,2025-08-26,paper,0.25,tcgplayer,retail,0.25


In [4]:
cardsDf = dfm3[['name', 'setName', 'setCode', 'releaseDate', 'language', 'types', 'colors', 'rarity', 'gameAvailability', 'uuid']]
pricesDf = dfm3[['date', 'cardFinish', 'price', 'priceProvider', 'avgMarketPrice', 'providerListing', 'uuid']]

cardsDf.to_sql('cards', conn, index=False, if_exists='replace')
pricesDf.to_sql('prices', conn, index=False, if_exists='replace')
conn.commit()

In [5]:
pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)

,name
0,cards
1,prices


### How many cards are printed per set?

In [17]:
query1 = """
SELECT
    c.setname,
    COUNT(DISTINCT c.uuid) as numCards,
    strftime('%Y', c.releaseDate) AS releaseYear
FROM cards c
GROUP BY c.setname
ORDER BY numCards DESC;
"""

result1 = pd.read_sql(query1, conn)
result1

,setName,numCards,releaseYear
0,The List,5024,2020
1,Secret Lair Drop,1880,2019
2,Universes Beyond: Doctor Who,1160,2023
3,Fallout,1052,2024
4,Commander Masters,1036,2023
...,...,...,...
606,Dragon Con,1,1994
607,Darksteel Promos,1,2004
608,Conspiracy Promos,1,2014
609,Coldsnap Promos,1,2006


In [ ]:
query8 = """
SELECT
    c.name,
    c.setname,
    p.price,
    p.priceProvider,
    p.avgMarketPrice,
    p.cardFinish,
    c.uuid
FROM cards c
JOIN prices p ON c.uuid = p.uuid
WHERE c.name = 'The Mary Janes'
ORDER BY p.price;
"""

result8 = pd.read_sql(query1, conn)
result8

,name,setName,price,priceProvider,avgMarketPrice,cardFinish,uuid
0,The Mary Janes,Marvel's Spider-Man Eternal,6.0,tcgplayer,6.0,normal,700e8ca0-f042-5e4b-bd80-85f46a2e0b23
1,The Mary Janes,Marvel's Spider-Man Eternal,6.0,tcgplayer,6.0,normal,700e8ca0-f042-5e4b-bd80-85f46a2e0b23
2,The Mary Janes,Marvel's Spider-Man Eternal,8.0,tcgplayer,8.0,foil,700e8ca0-f042-5e4b-bd80-85f46a2e0b23
3,The Mary Janes,Marvel's Spider-Man Eternal,8.0,tcgplayer,8.0,foil,700e8ca0-f042-5e4b-bd80-85f46a2e0b23
